### THIS SHIT WORKS.
##### Don't change it.

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from keras import layers

INPUT_SIZE = 10
LEARNING_RATE = 0.005

input = layers.Input(shape=(INPUT_SIZE,))
x = layers.Dense(10, activation='relu')(input)
x = layers.Dense(1, activation='tanh')(x)
F_curr_model = keras.Model(inputs=input, outputs=x)
F_prev_model = keras.Model(inputs=input, outputs=x)
F_curr_model.summary()

F_curr_model.compile(optimizer=keras.optimizers.Adam(LEARNING_RATE), loss='mse')

In [ ]:
def gen_price_series(size=10000, k=3, a=0.9):
    p_series = np.ndarray((size,))
    b_series = np.ndarray((size,))
    p_series[0] = 0
    b_series[0] = 0

    for i in range(1, size):
        p_series[i] =  p_series[i - 1] + b_series[i - 1] + k * np.random.normal()
        b_series[i] = a * b_series[i - 1] + np.random.normal()

    R = np.max(p_series) - np.min(p_series)
    z_series = np.exp(p_series / R)

    return z_series

def calc_price_returns(zt):
    # returns the set rt, with the first element (0) being NaN
    rt = np.ndarray((zt.shape[0],))
    rt[0] = np.nan
    rt[1:] = zt[1:] - zt[:-1]
    return rt


def calc_return(mu, rt, Ft_curr, Ft_prev, rft=0, delta=0):
    '''Calculates the returns (Rt) on-line.'''

    return mu * (rft + Ft_prev * (rt - rft) - delta * tf.math.abs(Ft_curr - Ft_prev))

def calc_DSR(n, Rt, At_prev, Bt_prev):
    '''Calculates the differential Sharpe ratios (DSR, Dt) on-line.'''

    At_curr = At_prev + n * (Rt - At_prev)
    Bt_curr = Bt_prev + n * (Rt ** 2 - Bt_prev)

    dDt_dRt = (Bt_prev - At_prev * Rt) / ((Bt_prev - At_prev ** 2) ** 1.5)

    return dDt_dRt, At_curr, Bt_curr

In [ ]:
def test_performance(zt, Ft):
    values = np.ones((zt.shape[0],))
    owned = False
    money = 1
    values[0] = money

    values_ideal = np.ones((zt.shape[0],))
    owned_ideal = False
    money_ideal = 1
    values_ideal[0] = money_ideal

    for t in range(INPUT_SIZE, SERIES_LENGTH - 1):

        # Model Ft
        if (Ft[t] > 0) and (not owned):
            # BUY
            owned = True
            money -= zt[t]
        elif (Ft[t] < 0) and owned:
            # SELL
            owned = False
            money += zt[t]
        
        # Ideal
        if (zt[t + 1] > zt[t]) and (not owned_ideal):
            # BUY
            owned_ideal = True
            money_ideal -= zt[t]
        elif (zt[t + 1] < zt[t]) and owned_ideal:
            # SELL
            owned_ideal = False
            money_ideal += zt[t]

        if owned:
            values[t] = money + zt[t]
        else:
            values[t] = money

        if owned_ideal:
            values_ideal[t] = money_ideal + zt[t]
        else:
            values_ideal[t] = money_ideal
    
    return (values[-1] / zt[-1], values_ideal[-1] / zt[-1]), (values, values_ideal)


In [ ]:
# GENERATION VARS

import time

MU = 3
N = 0.01
RISKFREE_RETURN = 0
TRANS_COST = 0.1
SERIES_LENGTH = 1000
TRADING_DELAY = 100
K = 3
A = 0.9

EPISODES = 1000

BASELINE_SERIES = gen_price_series(size=SERIES_LENGTH, k=K, a=A)

for ep in range(EPISODES):

    # generate price series
    zt = gen_price_series(size=SERIES_LENGTH, k=K, a=A)
    rt = calc_price_returns(zt)
    Ft = np.zeros((SERIES_LENGTH,))

    # breaks if init at one; MUST INIT ZERO
    At = np.zeros((SERIES_LENGTH,))
    Bt = np.zeros((SERIES_LENGTH,))

    SR_series = np.zeros((SERIES_LENGTH,))
    DSR_series = np.zeros((SERIES_LENGTH,))
    Rt_series = np.zeros((SERIES_LENGTH,))
    dD_series = np.ones((SERIES_LENGTH,))
    autodiff_series = np.ones((SERIES_LENGTH,))


    # plt.plot(zt)
    # plt.show()


    for t in range(INPUT_SIZE, SERIES_LENGTH):
        # Ta = time.time()

        with tf.GradientTape(persistent=True) as tape:
            # prediction, set new position
            F_curr = F_curr_model(zt[t - INPUT_SIZE + 1:t + 1].reshape(1, INPUT_SIZE))
            F_prev = F_prev_model(zt[t - INPUT_SIZE:t].reshape(1, INPUT_SIZE)) # On the first iteration, this does not exist and is not used.
            Ft[t] = F_curr
            print(F_curr)

            # if the first iteration, F(t-1) does not yet exist so no update can be made.
            # F_prev_model.set_weights(F_curr_model.get_weights())
            # calculate the gradient
            Rt = calc_return(MU, rt[t], F_curr, F_prev)
            dDt_dRt, At[t], Bt[t] = calc_DSR(N, Rt, At[t - 1], Bt[t - 1])
            
            Rt_series[t] = Rt
            SR_series[t] = np.mean(Rt_series[INPUT_SIZE:t]) / np.std(Rt_series[INPUT_SIZE:t])
            DSR = (Bt[t-1] * (Rt - At[t-1]) - (1/2) * (At[t-1] * (Rt ** 2 - Bt[t-1]))) / ((Bt[t-1] - At[t-1] ** 2) ** 1.5)
            DSR_series[t] = (Bt[t-1] * (Rt - At[t-1]) - (1/2) * (At[t-1] * (Rt ** 2 - Bt[t-1]))) / ((Bt[t-1] - At[t-1] ** 2) ** 1.5)
            dD_series[t] = dDt_dRt

        # calculate derivatives.
        Tgrad = time.time()
        dRt_dFcurr = tape.gradient(Rt, F_curr)
        dRt_dFprev = tape.gradient(Rt, F_prev)
        dFcurr_dThetacurr = tape.gradient(F_curr, F_curr_model.trainable_variables)
        dFprev_dThetaprev = tape.gradient(F_prev, F_prev_model.trainable_variables)
        autodiff_series[t] = tape.gradient(DSR, Rt)
        # print("Time taken for gradient: ", time.time() - Tgrad)

        # print(dRt_dFcurr.shape)
        # print(dFcurr_dThetacurr[0].shape)

        # dont update parameters if DSR hasn't stabilized yet
        if t < TRADING_DELAY:
            continue

        # Set F(t-1) to F(t) for the next iteration.
        F_prev_model.set_weights(F_curr_model.get_weights())

        # print(len(F_curr_model.trainable_variables))
        
        if t != INPUT_SIZE:
            # print("\nSAAAAA========================")
            # multiply derivatives together.
            gradient_update = []
            for i in range(len(dFcurr_dThetacurr)):
                grad = dDt_dRt * (dRt_dFcurr * dFcurr_dThetacurr[i] + dRt_dFprev * dFprev_dThetaprev[i])
                grad *= LEARNING_RATE
                # print(grad.shape)

                gradient_update.append(tf.reshape(grad, F_curr_model.trainable_variables[i].shape))


            VERBOSE = False
            if VERBOSE:
                X = -1
                print("\nSAAAAA========================")
                # a = F_curr_model.trainable_variables[X].numpy()
                # print("bias variable:", F_curr_model.trainable_variables[X])
                # print(F_curr_model.trainable_variables[-2])
                # print("grad update:", gradient_update[X])

                # print("grad update:", gradient_update[X].numpy())
                # print("grad update:", gradient_update)
            # if np.isnan(gradient_update[X].numpy()):
            #     dasufgapg = 1

            # F_curr_model.optimizer.apply_gradients(zip(gradient_update, F_curr_model.trainable_variables))

            vars = F_curr_model.trainable_variables
            for i in range(len(vars)):
                vars[i].assign_add(gradient_update[i])

            if VERBOSE:
                # b = F_curr_model.trainable_variables[X].numpy()
                # print("delta:", b - a)
                # print("new bias :", F_curr_model.trainable_variables[X])
                # print("output:", F_curr)
                print("BASELINE output:", F_curr_model(BASELINE_SERIES[-INPUT_SIZE:].reshape(1, INPUT_SIZE)))
            
            # print(Bt[t])

        # Tb = time.time()
        # print("Time taken: ", Tb - Ta)

    if ep % 1 == 0:
        print("Episode: ", ep)
        # test performance
        deltas, val_series = test_performance(zt, Ft)

        fig, ax = plt.subplots(3, 2, figsize=(15, 6))

        ax[0, 0].plot(zt)
        ax[0, 0].plot(val_series[0])
        ax[0, 0].set_title("price")

        ax[1, 0].plot(SR_series[:])
        ax[1, 0].set_title("SR")
        # plt.plot(rt)
        # plt.show()
        # plt.plot(Ft)
        # plt.show()
        # plt.plot(Rt_series)
        # plt.show()
        ax[0, 1].plot(DSR_series[:])
        ax[0, 1].set_title("DSR")

        ax[1, 1].plot(dD_series[:])
        ax[1, 1].set_title("dD/dR")
        ax[2, 1].plot(autodiff_series[:])
        ax[2, 1].set_title("autodiff_series")
        plt.show()


    # plt.plot(val_series[1])


In [ ]:
zt = gen_price_series(size=SERIES_LENGTH, k=K, a=A)
rt = calc_price_returns(zt)

print(zt[-10:])
print(rt[-10:])